Data comes from https://github.com/AMLab-Amsterdam/CEVAE/tree/master/datasets/TWINS

In [1]:

import pandas as pd
import json
from tqdm import tqdm

In [2]:
df_t = pd.read_csv("twin_pairs_T_3years_samesex.csv")
df_c_ = pd.read_csv("twin_pairs_X_3years_samesex.csv")
df_o = pd.read_csv("twin_pairs_Y_3years_samesex.csv")

In [3]:
df_c = df_c_[df_c_.columns[2:]].drop(columns=["infant_id_0", "infant_id_1"]).dropna()
df_c

,pldel,birattnd,brstate,stoccfipb,mager8,ormoth,mrace,meduc6,dmar,mplbir,...,nprevistq,dfageq,feduc6,dlivord_min,dtotord_min,bord_0,bord_1,brstate_reg,stoccfipb_reg,mplbir_reg
3,1.0,1.0,1,1,4,0.0,2,1.0,1,1.0,...,1.0,6.0,1.0,4.0,4.0,1.0,2.0,5,5,5.0
6,1.0,1.0,1,1,3,0.0,1,3.0,1,1.0,...,3.0,0.0,3.0,1.0,2.0,1.0,2.0,5,5,5.0
8,1.0,1.0,1,1,4,0.0,1,4.0,1,28.0,...,3.0,3.0,4.0,1.0,1.0,1.0,2.0,5,5,3.0
9,1.0,1.0,1,1,3,0.0,1,3.0,1,1.0,...,0.0,1.0,3.0,3.0,3.0,2.0,1.0,5,5,5.0
13,1.0,1.0,1,1,5,0.0,1,4.0,1,1.0,...,4.0,6.0,4.0,1.0,1.0,1.0,2.0,5,5,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71339,1.0,1.0,51,56,2,0.0,1,2.0,1,51.0,...,3.0,0.0,4.0,2.0,2.0,2.0,1.0,7,7,7.0
71340,1.0,1.0,51,56,4,0.0,3,4.0,0,51.0,...,1.0,0.0,3.0,5.0,5.0,2.0,1.0,7,7,7.0
71341,1.0,1.0,51,56,4,0.0,3,3.0,0,51.0,...,0.0,5.0,2.0,6.0,7.0,2.0,1.0,7,7,7.0
71342,1.0,1.0,51,56,2,0.0,1,2.0,1,38.0,...,0.0,0.0,2.0,1.0,1.0,1.0,2.0,7,7,8.0


In [4]:
shared_covs = [c for c in df_c.columns if not "_0" in c and not "_1" in c]
individual_covs = sorted(set([c[:-2] for c in df_c.columns if c.endswith(("_0", "_1"))]))

In [5]:
rows = []
pbar = tqdm(total=len(df_c))
for i, row in df_c.iterrows():
    mort_0, mort_1 = df_o.loc[i][["mort_0", "mort_1"]]
    rows.append([row[c] for c in shared_covs] + [0, mort_0])
    rows.append([row[c] for c in shared_covs] + [1, mort_1])
    pbar.update(1)
pbar.close()
df_compiled = pd.DataFrame(rows, columns=shared_covs + ["selection", "mortality"]).astype(int)

100%|██████████| 32120/32120 [00:28<00:00, 1136.66it/s]


In [7]:
display(df_compiled)
df_compiled.to_csv("twins.csv", index=False)

,pldel,birattnd,brstate,stoccfipb,mager8,ormoth,mrace,meduc6,dmar,mplbir,...,nprevistq,dfageq,feduc6,dlivord_min,dtotord_min,brstate_reg,stoccfipb_reg,mplbir_reg,selection,mortality
0,1,1,1,1,4,0,2,1,1,1,...,1,6,1,4,4,5,5,5,0,0
1,1,1,1,1,4,0,2,1,1,1,...,1,6,1,4,4,5,5,5,1,0
2,1,1,1,1,3,0,1,3,1,1,...,3,0,3,1,2,5,5,5,0,0
3,1,1,1,1,3,0,1,3,1,1,...,3,0,3,1,2,5,5,5,1,0
4,1,1,1,1,4,0,1,4,1,28,...,3,3,4,1,1,5,5,3,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64235,1,1,51,56,4,0,3,3,0,51,...,0,5,2,6,7,7,7,7,1,0
64236,1,1,51,56,2,0,1,2,1,38,...,0,0,2,1,1,7,7,8,0,0
64237,1,1,51,56,2,0,1,2,1,38,...,0,0,2,1,1,7,7,8,1,0
64238,1,1,51,56,5,0,1,5,1,6,...,1,4,5,3,3,7,7,7,0,0
